In [ ]:
from pasteur.kedro.ipython import * # type: ignore
register_kedro() # type: ignore

%load_ext autoreload
%autoreload 2

INFO     Kedro project Pasteur                                                                                           __init__.py:77
INFO     Defined global variable 'context', 'session', 'catalog' and 'pipelines'                                         __init__.py:78
INFO     Registered line magic 'run_viz'                                                                                 __init__.py:84


In [ ]:
# Use the first partition of texas_charges with the fit transformer
attrs = catalog.load('texas_billion.trn.table')['idx'].get_attributes()
wrk = catalog.load('texas_billion.wrk.idx_table')

import pandas as pd
import numpy as np

In [ ]:
from pasteur.hierarchy import rebalance_attributes
from pasteur.marginal import MarginalOracle
from pasteur.utils import LazyDataset, LazyPartition

m = MarginalOracle(attrs, wrk)
counts = m.get_counts()
m.close()

new_attrs = rebalance_attributes(counts, attrs, fixed=[2, 4, 8, 16, 32, 48], u=4)


WARNING  Launching a process pool implicitly. Use `init_pool()` to explicitly control pool creation.                    progress.py:227
WARNING  Rebalancing columns without using Differential Privacy (e_p=inf)                                              hierarchy.py:440


In [ ]:
Ns = [
    # 1,
    # 10,
    # 100,
    # 1_000,
    # 2_000,
    5_000,
    10_000,
    # 20_000
]

In [ ]:
for name, attr in new_attrs.items():
    for val_name, val in attr.vals.items():
        print(f"{val_name:>25s} | {', '.join(f'{h}:{val.get_domain(h):3d}' for h in range(val.height))}")


             revenue_code | 0:362, 1:192, 2: 48, 3: 32, 4: 16, 5:  8, 6:  4, 7:  3
          hcpcs_qualifier | 0:  5, 1:  4, 2:  3
     hcpcs_procedure_code | 0:3000, 1:768, 2:192, 3: 48, 4: 32, 5: 16, 6:  8, 7:  4, 8:  3
               modifier_1 | 0: 63, 1: 48, 2: 32, 3: 16, 4:  8, 5:  4, 6:  3
               modifier_2 | 0: 27, 1: 16, 2:  8, 3:  4, 4:  3
    unit_measurement_code | 0:  5, 1:  4, 2:  3
         units_of_service | 0: 21, 1: 16, 2:  8, 3:  4, 4:  2
                unit_rate | 0: 21, 1: 16, 2:  8, 3:  4, 4:  2
          chrgs_line_item | 0: 20, 1: 16, 2:  8, 3:  4, 4:  2
            chrgs_non_cov | 0: 20, 1: 16, 2:  8, 3:  4, 4:  2


In [ ]:
typ_1 = {'revenue_code': 7, 'hcpcs_qualifier': 2, 'hcpcs_procedure_code': 8, 'modifier_1': 6, 'modifier_2': 4, 'unit_measurement_code': 1, 'units_of_service': 3, 'unit_rate': 3, 'chrgs_line_item': 3}
tst = typ_1

def get_dom(cols):
    dom = 1
    for n, h in cols.items():
        dom *= new_attrs[n][n].get_domain(h)
    return dom

print(f'{get_dom(tst):,}')

62,208


In [ ]:
from pasteur.marginal import MarginalRequest, AttrSelector
from pasteur.utils.progress import init_pool
import time

req = MarginalRequest(None, {n: AttrSelector(n, 0, {n: h}) for n, h in typ_1.items()}, False)

def bench_oracle(oracle: MarginalOracle, workers: int, desc: str):
    with init_pool(workers):
        print(f"########\n{desc}, workers={workers}")
        for N in Ns:
            reqs = [req for _ in range(N)]

            start = time.perf_counter()
            oracle.process(reqs)
            end = time.perf_counter()

            duration = end - start
            print(f"N={N:10d}: {int(duration // 60)}:{duration % 60:06.3f}")

In [ ]:
inmemory = MarginalOracle(new_attrs, wrk, batched=False)

start = time.perf_counter()
inmemory._load_batch(wrk)
inmemory._loaded = True
end = time.perf_counter()

print(f"Inmemory load: {end - start:.2f}")

In [ ]:
duration = end - start
print(f"{int(duration // 60)}:{duration % 60:.3f}")
nbytes = inmemory.mem_cols.size + inmemory.mem_noncommon.size
print(f"Total memory use: {nbytes//1_000_000:,} MB")

In [ ]:
bench_oracle(inmemory, 1, "Inmemory")

In [ ]:
inmemory.min_chunk_size = 1
bench_oracle(inmemory, 32, "Inmemory")

In [ ]:
inmemory.close()

In [ ]:
parallel = MarginalOracle(new_attrs, wrk, batched=True, sequential_min=100_000)
bench_oracle(parallel, 32, "Parallel")
parallel.close()

In [ ]:
sequential = MarginalOracle(new_attrs, wrk, batched=True, sequential_min=0)
bench_oracle(sequential, 32, "Sequential")
sequential.close()

In [ ]:
parallel = MarginalOracle(new_attrs, wrk, batched=True, sequential_min=100_000)
bench_oracle(parallel, 1, "Parallel")
parallel.close()

In [ ]:
# sequential = MarginalOracle(new_attrs, wrk, batched=True, sequential_min=0)
# bench_oracle(sequential, 1, "Sequential")
# sequential.close()

In [ ]:
inmemory = MarginalOracle(new_attrs, wrk, batched=False)

start = time.perf_counter()
inmemory._load_batch(wrk)
inmemory._loaded = True
end = time.perf_counter()

print(f"Inmemory load: {end - start:.2f}")

In [ ]:
duration = end - start
print(f"{int(duration // 60)}:{duration % 60:.3f}")
nbytes = inmemory.mem_cols.size + inmemory.mem_noncommon.size
print(f"Total memory use: {nbytes//1_000_000:,} MB")

In [ ]:
inmemory._loaded = True
inmemory.min_chunk_size = 1

In [ ]:
bench_oracle(inmemory, 16, "Inmemory")

In [ ]:
inmemory.close()

In [ ]:
sequential.close()

In [ ]:
sequential = MarginalOracle(new_attrs, wrk, batched=True, sequential_min=0, max_worker_mult=2)
bench_oracle(sequential, 16, "Sequential")
sequential.close()

########
Sequential, workers=16
N=      5000: 37:05.007                                                                                                                           7.05s/it]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_3468248/2473194648.py:2 in <cell line: 2>                                         │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_3468248/2473194648.py'                      │
│                                                                                                  │
│ /tmp/ipykernel_3468248/1354924721.py:14 in bench_oracle                                          │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_3468248/1354924721.py'                      │
│                                                                                                  │
│ /srv/data/antheas/projects/pasteur/src/pasteur/marginal/__init__.py:337 in process               │
│                                                                                                  │
│   334 │   │   │   logger.debug(                                                                  │
│   335 │   │   │   │   f"Processing {len(requests)} marginals by sequential loading of partitio   │
│   336 │   │   │   )                                                                              │
│ ❱ 337 │   │   │   return self._process_batched_sequential(requests, desc)                        │
│   338 │   │                                                                                      │
│   339 │   │   logger.debug(                                                                      │
│   340 │   │   │   f"Processing {len(requests)} marginals by loading partitions in parallel."     │
│                                                                                                  │
│ /srv/data/antheas/projects/pasteur/src/pasteur/marginal/__init__.py:283 in                       │
│ _process_batched_sequential                                                                      │
│                                                                                                  │
│   280 │   │   │   if i < len(partitions) - 1:                                                    │
│   281 │   │   │   │   res = process_async(_marginal_loader, self.attrs, partitions[i + 1])       │
│   282 │   │   │                                                                                  │
│ ❱ 283 │   │   │   self._process_batch(requests)                                                  │
│   284 │   │   │   self._unload_batch()                                                           │
│   285 │   │                                                                                      │
│   286 │   │   assert self.mem_marginals is not None and self.info_marginals is not None          │
│                                                                                                  │
│ /srv/data/antheas/projects/pasteur/src/pasteur/marginal/__init__.py:188 in _process_batch        │
│                                                                                                  │
│   185 │   │                                                                                      │
│   186 │   │   per_call_args = [{"i": i, "req": req} for i, req in enumerate(requests)]           │
│   187 │   │                                                                                      │
│ ❱ 188 │   │   res = process_in_parallel(                                                         │
│   189 │   │   │   _marginal_worker,                                                              │
│   190 │   │   │   per_call_args,                                                                 │
│   191 │   │   │   base_args,                                                                     │
│                                                            

: 

In [ ]:
import tqdm
tqdm.tqdm._instances